In [1]:
from fastai.vision import *
from pathlib import Path
import pickle
import sys
PATH = Path('data')
list(PATH.iterdir())

[PosixPath('data/challenge-2019-classes-description-500.csv'),
 PosixPath('data/fulltest'),
 PosixPath('data/challenge-2019-validation-detection-human-imagelabels.csv'),
 PosixPath('data/Xclassdescription.csv'),
 PosixPath('data/challenge-2019-validation-detection-bbox.csv'),
 PosixPath('data/fulltrain'),
 PosixPath('data/challenge-2019-train-detection-bbox.csv'),
 PosixPath('data/fullvalidation'),
 PosixPath('data/challenge-2019-train-detection-human-imagelabels.csv')]

In [2]:
str(list(PATH.iterdir())[0])

'data/challenge-2019-classes-description-500.csv'

In [3]:
str(list(PATH.iterdir())[0])

'data/challenge-2019-classes-description-500.csv'

In [4]:
[item.name for item in list((PATH/'fulltrain').iterdir())[:10]]

['f8a757ad752b039c.jpg',
 'd1db1322f81fd14a.jpg',
 '5a08ec130d7e8432.jpg',
 'c12e0de516bcebe3.jpg',
 'ab673ab88d92186d.jpg',
 'fababfecbe82e36a.jpg',
 '9acedff8a02e68f7.jpg',
 '518491f3cf884c91.jpg',
 '3c8b4f6543f5c08f.jpg',
 '79baedadf3083ee4.jpg']

In [5]:
class_names = pd.read_csv(PATH/'challenge-2019-classes-description-500.csv', header=None, names=["LabelName", "Class"])

#### Labels, bounding boxes for training

In [25]:
il_train = ImageList.from_folder(PATH/"fulltrain/", convert_mode='L') # contains 1743042 images
bboxes_train = pd.read_csv(PATH/'challenge-2019-train-detection-bbox.csv')
labels_train = bboxes_train.merge(class_names, on="LabelName")
labels_train.loc[:, "ImageID"] = labels_train.loc[:, "ImageID"].astype('category')

1743042

In [8]:
ids = labels_train.ImageID.unique()
df_dict = labels_train.to_dict("list") 
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids}
[x.update({image_id : {"bboxes" : x[image_id]["bboxes"] + [[xmin, xmax, ymin, ymax]], "Classes" : x[image_id]["Classes"] + [c]}} ) 
     for c, image_id, xmin, xmax, ymin, ymax in zip(df_dict["Class"], df_dict["ImageID"], df_dict["XMin"], df_dict["XMax"], df_dict["YMin"], df_dict["YMax"])]
img2bbox_train = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids}

#### Labels, bounding boxes for validation

In [26]:
il_val = ImageList.from_folder(PATH/"fullvalidation/", convert_mode='L') # contains 41620 images
bboxes_val = pd.read_csv(PATH/'challenge-2019-validation-detection-bbox.csv')
labels_val = bboxes_val.merge(class_names, on="LabelName")
labels_val.loc[:, "ImageID"] = labels_val.loc[:, "ImageID"].astype('category')

41620

In [10]:
ids = labels_val.ImageID.unique()
df_dict = labels_val.to_dict("list") 
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids}
[x.update({image_id : {"bboxes" : x[image_id]["bboxes"] + [[xmin, xmax, ymin, ymax]], "Classes" : x[image_id]["Classes"] + [c]}} ) 
     for c, image_id, xmin, xmax, ymin, ymax in zip(df_dict["Class"], df_dict["ImageID"], df_dict["XMin"], df_dict["XMax"], df_dict["YMin"], df_dict["YMax"])]
img2bbox_val = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids}

In [11]:
img2bbox = {**img2bbox_train, **img2bbox_val}

In [42]:
print(len(il_train))
print(len(img2bbox_train.keys()))
print(len(il_val))
print(len(img2bbox_val.keys()))
print(len(list(img2bbox.keys())))
print(len(il_train)-len(il_val))
print('IST')
print(len(img2bbox.keys()))
print('SOLL')
print(len(il_train)+len(il_val))
print('SOLL-IST')

1743042
1674979
41620
34917
1709896
1701422
IST
1709896
SOLL
1784662


In [65]:
i = 1
def get_y_func(o):
    try:
        return img2bbox[o.stem]
    except:
        global i
        #print(o, i)
        i += 1
get_y_func = lambda o:img2bbox[o.stem]

In [ ]:
il_full = list(il_train) + list(il_val)

In [ ]:
print(il_full[0], type(il_full[0]))

In [66]:
data = (
    ObjectItemList.from_folder(PATH/'fulltrain', convert_mode='L')
    .split_by_rand_pct(0.2)
    .label_from_func(get_y_func)
)

KeyError: 'f126539d4b1e6a42'

In [49]:
def get_data(bs, size):
    src = ImageList.from_folder(PATH/'fulltrain', convert_mode='L')
    src
    # src = src.split_by_files(il_val_sub)
    src = src.split_by_files(img2bbox_val.keys())
    src = src.label_from_func(get_y_func)
    return src.databunch(path=PATH, bs=bs, collate_fn=bb_pad_collate)

In [50]:
data = get_data(4, 128)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/fastai/data_block.py:445: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


TypeError: unhashable type: 'list'

In [ ]:
with open('train_val_data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [43]:
data

NameError: name 'data' is not defined